In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install -U groq
!pip install tavily-python
!pip install firecrawl-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.1/191.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import httpx
from typing import List, Dict, Any
from groq import Groq
from datetime import datetime
from tavily import TavilyClient
from firecrawl import FirecrawlApp


In [ ]:
class Tool:
    def __init__(self, name: str, func: callable, description: str):
        self.name = name
        self.func = func
        self.description = description

    def run(self, input_q):
        return self.func(input_q)

In [ ]:
class ReActAgent:
    def __init__(self, client: Groq, system: str, tools: List[Tool]):
        self.client = client
        self.system = system
        self.tools = tools
        self.messages: List[Dict[str, str]] = [{"role": "system", "content": system}]
        self.max_iterations = 3

    def _get_completion(self):
        completion = self.client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=self.messages,
            temperature=0.7
        )
        return completion.choices[0].message.content

    def run(self, query: str):
        self.messages.append({"role": "user", "content": query})

        for iteration in range(self.max_iterations):
            response = self._get_completion()
            print(f"\n{'='*60}")
            print(f"Iteration {iteration + 1}")
            print(f"{'='*60}")
            print(f"Agent Response:\n{response}")
            print(f"{'='*60}")


            if "Answer:" in response:
                # Add to history
                self.messages.append({"role": "assistant", "content": response})
                answer = response.split("Answer:", 1)[1].strip() #Split on the first occurance of 'Answer' take the [1]
                return answer

            # Updated regex to match Action: ToolName["input"] or Action: ToolName[input]
            action_match = re.search(
                r"Action:\s*(\w+)\[\"?([^\"[\]]*)\"?\]",
                response,
                re.IGNORECASE | re.DOTALL
            )

            if action_match:
                tool_name = action_match.group(1).strip()
                tool_input = action_match.group(2).strip()

                # Add the assistant's thought + action to history
                self.messages.append({"role": "assistant", "content": response})

                print(f"\n🔧 Executing Tool: {tool_name}")
                print(f"📥 Tool Input: {tool_input}")

                observation = self._execute_tool(tool_name, tool_input)

                print(f"📤 Tool Output: {observation[:500]}...")  # Print first 300 chars

                # Add observation to conversation - this prompts the next iteration
                obs_message = f"Observation: {observation}\n\nNow continue your reasoning."
                self.messages.append({"role": "user", "content": obs_message})
            else:
                # No action found - add response and prompt for action
                print("⚠️ No valid action found in response. Prompting for action...")
                self.messages.append({"role": "assistant", "content": response})
                self.messages.append({
                    "role": "user",
                    "content": "Please provide an Action using one of the available tools, or give your final Answer."
                })

        print("\nMax iterations reached. Attempting a final answer based on available information...")

        final_input = " ".join(msg['content'] for msg in self.messages)


        self.messages.append({"role": "user", "content": final_input + "\n\nBased on the information above, please provide a final answer."})
        final_response = self._get_completion()

        if "Answer:" in final_response:
            final_answer = final_response.split("Answer:", 1)[1].strip()
            return final_answer

        return "Unable to provide a final answer from the available context."

    def _execute_tool(self, tool_name: str, tool_input: str):
        # Find and execute the matching tool
        for tool in self.tools:
            if tool.name.lower() == tool_name.lower():
                try:
                    # Pass input directly without wrapping in quotes

                      return tool.run(tool_input)
                except Exception as e:
                    return f"Error executing {tool_name}: {str(e)}"

        return f"Tool '{tool_name}' not found. Available tools: {', '.join([t.name for t in self.tools])}"


In [ ]:
def clean_html(text):
    text = re.sub(r"<.*?>", "", text)  # remove tags
    text = text.replace("&#039;", "'")  # unescape apostrophe
    text = text.replace("&quot;", "\"")
    text = text.replace("&amp;", "&")
    return text

In [ ]:
def wikipedia_search(query: str):
    try:
        response = httpx.get("https://en.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": query,
            "format": "json"
        },
        headers={"User-Agent": "MyLLMAgent/1.0 (sjbzmn67@gmail.com)"},
        timeout=10.0)

        print(response)
        results = response.json()["query"]["search"]
        if results:
            return clean_html(results[0]["snippet"])
        return "No results found"
    except Exception as e:
        return f"Wikipedia search error: {str(e)}"

def calculate(expression: str):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Calculation error: {str(e)}"

def current_date(query: str):
    return datetime.now().strftime("%Y-%m-%d")

def tavily_search(query: str, api_key: str):
    try:
        tavily = TavilyClient(api_key=api_key)
        result = tavily.search(
            query=query,
            search_depth="advanced",
            max_results=2,
            include_answer=False,
            include_raw_content=False,
            include_images=False
        )
        results = result.get("results", [])
        if results:
            return "\n\n".join([f"{r['content']}" for r in results])
        return "No results found"
    except Exception as e:
        return f"Tavily search error: {str(e)}"

def firecrawl_search(query: str, api_key: str):
    try:
        r = httpx.post(
            "https://api.firecrawl.dev/v2/search",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "sources": ["web"],
                "categories": [],
                "limit": 1,
                "query": query,
                "scrapeOptions": {
                    "onlyMainContent": False,
                    "maxAge": 172800000,
                    "parsers": ["pdf"],
                    "formats": []
                }
            },
            timeout=15.0
        )
        data = r.json()
        if 'data' in data and 'web' in data['data'] and data['data']['web']:
            return data['data']['web'][0].get('description', 'No description available')
        return "No results found"
    except Exception as e:
        return f"Firecrawl search error: {str(e)}"


def get_weather(location: str, api_key: str):
      try:
        # Get weather data
        response = httpx.get(
            "https://api.openweathermap.org/data/2.5/weather",
            params={
                "q": location,
                "appid": api_key,
                "units": "metric"  # Use "imperial" for Fahrenheit
            },
            timeout=10.0
        )

        if response.status_code != 200:
            return f"Weather API error: {response.status_code} - {response.text}"

        data = response.json()

        # Extract relevant information
        city = data.get("name", "Unknown")
        country = data.get("sys", {}).get("country", "")
        temp = data.get("main", {}).get("temp", "N/A")
        feels_like = data.get("main", {}).get("feels_like", "N/A")
        humidity = data.get("main", {}).get("humidity", "N/A")
        description = data.get("weather", [{}])[0].get("description", "N/A")
        wind_speed = data.get("wind", {}).get("speed", "N/A")

        # Format the response
        weather_info = f"""Weather for {city}, {country}:
        - Temperature: {temp}°C (feels like {feels_like}°C)
        - Conditions: {description}
        - Humidity: {humidity}%
        - Wind Speed: {wind_speed} m/s"""

        return weather_info

      except Exception as e:
        return f"Weather lookup error: {str(e)}"

In [ ]:
system_prompt = """You are an AI assistant that uses the ReAct pattern to answer questions.

CRITICAL RULES:
1. You can only take ONE action at a time
2. After each Action, you MUST wait for an Observation
3. Do NOT write "(wait for observation)" - the system will provide it
4. Do NOT continue reasoning after an Action - STOP and wait

For each turn, do EXACTLY ONE of these:
- Give a Thought + Action (then STOP). If you don't think an action is necessary, go for the final Answer immediately.
- Give a final Answer, once you receive an answer from any of the tools

Available tools:

- Calculate[expression]: Do math
- CurrentDate[]: Get today's date
- TavilySearch[query]: Web search for latest/current/recent information
- FireCrawlSearch[query]: Web search for latest/current/recent information
- GetWeather[location]: Get current weather (e.g., "London", "New York,US", "Tokyo,JP")

IMPORTANT FORMAT RULES:
- Write actions as: Action: ToolName[your query here]
- Example: Action: TavilySearch[Who won 2025 NYC mayor election]
- Example: Action: GetWeather[London]
- When you have the probable answer from tool actions, write: Answer: [your final answer]
- Give your output in json format:
```
{

}
```

Example:
User: What's the weather like in Paris?

Thought: I need to check the current weather in Paris.
Action: GetWeather[Paris,FR]

(wait for observation)

Thought: Based on the weather data...
Answer: [provide answer]
"""

In [ ]:
def create_agent(groq_api_key: str, tavily_api_key: str, firecrawl_api_key: str, weather_api_key: str):
    client = Groq(api_key=groq_api_key)

    tools = [
        #Tool("Wikipedia", wikipedia_search, "Search Wikipedia for information"),
        Tool("Calculate", calculate, "Perform mathematical calculations"),
        Tool("CurrentDate", current_date, "Get the current date"),
        Tool("TavilySearch", lambda q: tavily_search(q, tavily_api_key), "Search web for latest information"),
        Tool("FireCrawlSearch", lambda q: firecrawl_search(q, firecrawl_api_key), "Search web for recent information"),
        Tool("GetWeather", lambda q: get_weather(q, weather_api_key), "Get current weather for a location"),
    ]

    return ReActAgent(client=client, system=system_prompt, tools=tools)


In [ ]:
from google.colab import userdata

In [ ]:

agent = create_agent(
    groq_api_key=userdata.get('Groq'),
    tavily_api_key=userdata.get('Tavily'),
    firecrawl_api_key=userdata.get('Firecrawl'),
    weather_api_key=userdata.get('OpenWeatherMap')
)
result = agent.run("Who won the 2025 NYC mayor election which happened last month?")
print(f"\n{'='*60}")
print(f"FINAL ANSWER: {result}")
print(f"{'='*60}")


Iteration 1
Agent Response:
Thought: I need to find the latest news on the 2025 NYC mayor election.
Action: TavilySearch[2025 NYC mayoral election winner]

🔧 Executing Tool: TavilySearch
📥 Tool Input: 2025 NYC mayoral election winner
📤 Tool Output: An election for the mayor of New York City was held on November 4, 2025. Democratic "Democratic Party (United States)") state assemblyman Zohran Mamdani won the election with 50.4% of the vote, defeating Republican "Republican Party (United States)") activist Curtis Sliwa and independent former Democratic governor Andrew Cuomo.( This election featured the highest turnout for a New York City mayoral election in decades, mainly due to significant young voter registration.( Mamdani is scheduled to ...

Iteration 2
Agent Response:
Thought: I have found the result of the 2025 NYC mayoral election, but the winner is not explicitly mentioned as the final answer. However, based on the information provided, it seems that Zohran Mamdani won the elect

In [ ]:
agent = create_agent(
    groq_api_key=userdata.get('Groq'),
    tavily_api_key=userdata.get('Tavily'),
    firecrawl_api_key=userdata.get('Firecrawl'),
    weather_api_key=userdata.get('OpenWeatherMap')
)
result = agent.run("What is the name of Iphone version released in 2025?")
print(f"\n{'='*60}")
print(f"FINAL ANSWER: {result}")
print(f"{'='*60}")


Iteration 1
Agent Response:
Thought: I need to find the latest information on iPhone releases in 2025.
Action: TavilySearch[Iphone release 2025]

🔧 Executing Tool: TavilySearch
📥 Tool Input: Iphone release 2025
📤 Tool Output: Apple’s 2025 iPhone lineup was announced at Apple’s “Awe Dropping” event on September 9, 2025.
 There are three new models including a brand-new iPhone category: the iPhone Air, the slimmest iPhone ever, at just 5.6mm thick (the previous record was 6.9mm with the iPhone 6).
 Pre-orders started on September 12, 2025. iPhones will begin shipping on September 19, 2025. [...] New iPhone models for 2025 were unveiled on Tuesday, September 9, 2025.
 Pre-orders started on Friday, September 12, 2025, wi...

Iteration 2
Agent Response:
Thought: I need to determine the name of the iPhone version released in 2025.
Observation: The text mentions the iPhone 17 series, which includes the iPhone 17, iPhone 17 Pro, iPhone 17 Pro Max, and the iPhone Air.
Answer: The iPhone 17 ser

In [ ]:
agent = create_agent(
    groq_api_key=userdata.get('Groq'),
    tavily_api_key=userdata.get('Tavily'),
    firecrawl_api_key=userdata.get('Firecrawl'),
    weather_api_key=userdata.get('OpenWeatherMap')
)
result = agent.run("What is the name of the scientist that discovered the theory of relativity?")
print(f"\n{'='*60}")
print(f"FINAL ANSWER: {result}")
print(f"{'='*60}")


Iteration 1
Agent Response:
Thought: I need to find information about the scientist who discovered the theory of relativity.
Action: Wikipedia[Scientist who discovered theory of relativity]

🔧 Executing Tool: Wikipedia
📥 Tool Input: Scientist who discovered theory of relativity
<Response [200 OK]>
📤 Tool Output: ideological bases. Though some <span class="searchmatch">of</span> these criticisms had the support <span class="searchmatch">of</span> reputable <span class="searchmatch">scientists</span>, Einstein&#039;s <span class="searchmatch">theory</span> <span class="searchmatch">of</span> <span class="searchmatch">relativity</span> is now accepted by the scientific...

Iteration 2
Agent Response:
Thought: The Wikipedia search result doesn't seem to directly answer my question about the scientist who discovered the theory of relativity.
Action: Wikipedia[Albert Einstein]

🔧 Executing Tool: Wikipedia
📥 Tool Input: Albert Einstein
<Response [200 OK]>
📤 Tool Output: <span class="searchma

In [ ]:
agent = create_agent(
    groq_api_key=userdata.get('Groq'),
    tavily_api_key=userdata.get('Tavily'),
    firecrawl_api_key=userdata.get('Firecrawl'),
    weather_api_key=userdata.get('OpenWeatherMap')
)
result = agent.run("What's the current temperature in London?")
print(f"\n{'='*60}")
print(f"FINAL ANSWER: {result}")
print(f"{'='*60}")


Iteration 1
Agent Response:
Thought: I need to check the current weather in London to get the temperature.
Action: GetWeather[London, UK]

🔧 Executing Tool: GetWeather
📥 Tool Input: London, UK
📤 Tool Output: Weather for London, GB:
        - Temperature: 11.51°C (feels like 11.09°C)
        - Conditions: overcast clouds
        - Humidity: 91%
        - Wind Speed: 7.2 m/s...

Iteration 2
Agent Response:
Thought: Since the temperature is given, that's the answer I'm looking for.
Answer: 11.51°C

FINAL ANSWER: 11.51°C


In [ ]:
agent = create_agent(
    groq_api_key=userdata.get('Groq'),
    tavily_api_key=userdata.get('Tavily'),
    firecrawl_api_key=userdata.get('Firecrawl'),
    weather_api_key=userdata.get('OpenWeatherMap')
)
result = agent.run("Can you tell me today's date?")
print(f"\n{'='*60}")
print(f"FINAL ANSWER: {result}")
print(f"{'='*60}")


Iteration 1
Agent Response:
Action: CurrentDate[]

🔧 Executing Tool: CurrentDate
📥 Tool Input: 
📤 Tool Output: 2025-12-01...

Iteration 2
Agent Response:
Thought: Since I have today's date, I don't need to proceed with any further actions.

Answer: Today's date is 2025-12-01.

FINAL ANSWER: Today's date is 2025-12-01.


In [ ]:
agent = create_agent(
    groq_api_key=userdata.get('Groq'),
    tavily_api_key=userdata.get('Tavily'),
    firecrawl_api_key=userdata.get('Firecrawl'),
    weather_api_key=userdata.get('OpenWeatherMap')
)
result = agent.run("Which university did the author of The Martian attend?")
print(f"\n{'='*60}")
print(f"FINAL ANSWER: {result}")
print(f"{'='*60}")


Iteration 1
Agent Response:
Thought: To find the information about the author of The Martian, I need to search for the author's background.
Action: TavilySearch[The Martian author education background]

🔧 Executing Tool: TavilySearch
📥 Tool Input: The Martian author education background
📤 Tool Output: Andy Weir, who describes himself on his Web site as a "lifelong space nerd," has been writing science fiction since his early twenties, publishing these stories on his Web site. His first novel, The Martian, also began as a self-published work, which Weir posted on his Web site in serialized form over several years. It proved so popular that Weir made it available as an e-book on Kindle. Its popularity led to a lucrative book deal for Weir. The Martian  reached number twelve on the New York [......

Iteration 2
Agent Response:
Thought: Based on the information provided, Andy Weir attended the University of California, San Diego, but I need to confirm this information.
Action: Calculate[U